#### 将数据转换为tf_Records
**转换的意义主要为方便数据的存储和读写操作,同时也可以加快训练速度**

In [ ]:
import random  # 为了后面随机化数据
import tensorflow as tf
import cv2 # 为了读取图像
import os

In [ ]:
#coding=utf-8


import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize
import scipy.io as sio  
import cv2

class vgg16:
    def __init__(self, imgs, weights=None, sess=None):
        self.imgs = imgs
        self.convlayers()
        self.fc_layers()
        self.probs = tf.nn.softmax(self.fc3l)
        if weights is not None and sess is not None:
            self.load_weights(weights, sess)


    def convlayers(self):
        self.parameters = []

        # zero-mean input
        with tf.name_scope('preprocess') as scope:
            mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
            images = self.imgs-mean

        # conv1_1
        with tf.name_scope('conv1_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv1_2
        with tf.name_scope('conv1_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool1
        self.pool1 = tf.nn.max_pool(self.conv1_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool1')

        # conv2_1
        with tf.name_scope('conv2_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv2_2
        with tf.name_scope('conv2_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool2
        self.pool2 = tf.nn.max_pool(self.conv2_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool2')

        # conv3_1
        with tf.name_scope('conv3_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_2
        with tf.name_scope('conv3_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_3
        with tf.name_scope('conv3_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool3
        self.pool3 = tf.nn.max_pool(self.conv3_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool3')

        # conv4_1
        with tf.name_scope('conv4_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool3, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_2
        with tf.name_scope('conv4_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_3
        with tf.name_scope('conv4_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool4
        self.pool4 = tf.nn.max_pool(self.conv4_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

        # conv5_1
        with tf.name_scope('conv5_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool4, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_2
        with tf.name_scope('conv5_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_3
        with tf.name_scope('conv5_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool5
        self.pool5 = tf.nn.max_pool(self.conv5_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

    def fc_layers(self):
        # fc1
        with tf.name_scope('fc1') as scope:
            shape = int(np.prod(self.pool5.get_shape()[1:]))
            fc1w = tf.Variable(tf.truncated_normal([shape, 4096],
                                                         dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
            fc1b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
                                 trainable=True, name='biases')
            pool5_flat = tf.reshape(self.pool5, [-1, shape])
            fc1l = tf.nn.bias_add(tf.matmul(pool5_flat, fc1w), fc1b)
            self.fc1 = tf.nn.relu(fc1l)
            self.parameters += [fc1w, fc1b]

        # fc2
        with tf.name_scope('fc2') as scope:
            fc2w = tf.Variable(tf.truncated_normal([4096, 4096],
                                                         dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
            fc2b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
                                 trainable=True, name='biases')
            fc2l = tf.nn.bias_add(tf.matmul(self.fc1, fc2w), fc2b)
            self.fc2 = tf.nn.relu(fc2l)
            self.parameters += [fc2w, fc2b]

        # fc3
        with tf.name_scope('fc3') as scope:
            fc3w = tf.Variable(tf.truncated_normal([4096, 1000],
                                                         dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
            fc3b = tf.Variable(tf.constant(1.0, shape=[1000], dtype=tf.float32),
                                 trainable=True, name='biases')
            self.fc3l = tf.nn.bias_add(tf.matmul(self.fc2, fc3w), fc3b)
            self.parameters += [fc3w, fc3b]

    def load_weights(self, weight_file, sess):
        weights = np.load(weight_file)
        keys = sorted(weights.keys())
        for i, k in enumerate(keys):
            print i, k, np.shape(weights[k])
            sess.run(self.parameters[i].assign(weights[k]))


def save_feature(feature_normal,file_path):
	
	import os
	if not os.path.exists(file_path):
		os.mkdir(file_path)

	shape=feature_normal.shape
	feature = np.reshape(feature_normal,[shape[1],shape[2],shape[3]])
	final_feature=feature[:,:,0]

	connect_Feature=np.zeros([shape[1]*shape[3],shape[2]])

	for i in range(shape[3]):
		final_feature+=feature[:,:,i]
		connect_Feature[i*shape[2]:(i+1)*shape[2],:]=feature[:,:,i]
		
	cv2.imwrite(file_path+"/"+"final_feature.jpg",final_feature/shape[3])

	cv2.imwrite(file_path+"/"+"connect_Feature.jpg",connect_Feature)


if __name__ == '__main__':

    sess = tf.Session()  

    img_name="test.jpg"
    model_name="vgg16_weights.npz"

    imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])  

    vgg = vgg16(imgs, model_name, sess)  
      
    img= imread(img_name, mode='RGB')

    img= imresize(img, (224, 224))   
      
    #img=np.ones([224,224,3])



    feature_fc3= sess.run(vgg.fc3l, feed_dict={vgg.imgs:[img]})

    print("feature_fc3:",feature_fc3.shape,feature_fc3)


    """
    feature_conv1_1,feature_conv1_2,feature_conv2_1,feature_conv2_2,feature_conv3_1,feature_conv3_2,feature_conv3_3,feature_conv4_1,feature_conv4_2,feature_conv4_3,feature_conv5_1,feature_conv5_2,feature_conv5_3= sess.run((vgg.conv1_1,vgg.conv1_2,vgg.conv2_1,vgg.conv2_2,vgg.conv3_1,vgg.conv3_2,vgg.conv3_3,vgg.conv4_1,vgg.conv4_2,vgg.conv4_3,vgg.conv5_1,vgg.conv5_2,vgg.conv5_3), feed_dict={vgg.imgs: [img]}) 


    save_feature(feature_conv1_1,"feature_conv1_1")
    save_feature(feature_conv1_2,"feature_conv1_2")
    save_feature(feature_conv2_1,"feature_conv2_1")
    save_feature(feature_conv2_2,"feature_conv2_2")
    save_feature(feature_conv3_1,"feature_conv3_1")
    save_feature(feature_conv3_2,"feature_conv3_2")
    save_feature(feature_conv3_3,"feature_conv3_3")
    save_feature(feature_conv4_1,"feature_conv4_1")
    save_feature(feature_conv4_2,"feature_conv4_2")
    save_feature(feature_conv4_3,"feature_conv4_3")
    save_feature(feature_conv5_1,"feature_conv5_1")
    save_feature(feature_conv5_2,"feature_conv5_2")
    save_feature(feature_conv5_3,"feature_conv5_3")

    """

#### 定义相关预处理函数，可完成数据转换工作

In [2]:
def _int64_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def _float_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def _bytes_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def _process_image_withoutcoder(filename):
    image = cv2.imread(filename)
    if not image is None:
        image_data = image.tostring()
        assert len(image.shape) == 3
        height = image.shape[0]
        width = image.shape[1]
        assert image.shape[2] == 3
        return image_data, height, width
    else:
        return None,None,None


def _convert_to_example_simple(image_example, image_buffer):
    """
    image_example
    image_buffer
    """
    class_label = image_example['label']
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'image/encoded': _bytes_feature(image_buffer),
        'image/label': _int64_feature(class_label),
    }))
    return example


def _add_to_tfrecord(filename, image_example, tfrecord_writer):
    image_data, _, _ = _process_image_withoutcoder(filename)
    if not image_data is None:
        example = _convert_to_example_simple(image_example, image_data)
        tfrecord_writer.write(example.SerializeToString())

In [3]:
def create_tf_records(data_dir,data_annotation_name):
    """
    用于产生生产tf_records
    
        data_dir
            -label_images1
            -label_images2
            
        train_data.txt   
            image_path label
            image_path label
        
    """
    print("start create tf records")
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    if not os.path.exists(data_annotation_name):
        print("data_annotation_name  is not exists")
     
    
    # 设置输入的tf_Records 路径
    file_output=os.path.join(data_dir,"train_data.tfrecord_shuffle")
    
    f_lines=open(data_annotation_name).readlines()
    dataset = []
    for line_index,f_line in enumerate(f_lines):
        info = f_line.strip().split()
        data_example = dict()
        bbox = dict()
        
        data_example["file_name"] = info[0] #路径
        
        data_example["label"] = int(info[1]) #label
        
        dataset.append(data_example)
        
        if (line_index+1)%5000==0:
            print("now process image num is : %d"%(line_index+1))
    
    # 数据扰动
    random.shuffle(dataset)
    
    with tf.python_io.TFRecordWriter(file_output) as tfrecord_writer:
        for index, image_example in enumerate(dataset):
            
            filename = os.path.join(data_dir,image_example['file_name'])
            
            _add_to_tfrecord(filename, image_example, tfrecord_writer)
            
    tfrecord_writer.close()
    
    print("finish write tf records")

In [8]:
data_dir="dataset"

data_annotation_name="dataset/train_data.txt"

create_tf_records(data_dir,data_annotation_name)

start create tf records
now process image num is : 5000
now process image num is : 10000
now process image num is : 15000
now process image num is : 20000
now process image num is : 25000
now process image num is : 30000
now process image num is : 35000
now process image num is : 40000
now process image num is : 45000
now process image num is : 50000
now process image num is : 55000
finish write tf records


####  用于tf records数据读取

In [23]:
def read_single_tfrecord(tfrecord_file, batch_size, image_size):
    """
    tfrecord_File:路径
    batch_size:我们一次从tfrecord中读取的数据量的大小
    image_size:这个一般就默认，我们存入的时候图像的大小是多大就多大
    
    width=height
    cv2.resize(img,(width,height))
    """
    filename_queue = tf.train.string_input_producer(
        [tfrecord_file], shuffle=True)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    image_features = tf.parse_single_example(
        serialized_example,
        features={
            'image/encoded': tf.FixedLenFeature([], tf.string),
            'image/label': tf.FixedLenFeature([], tf.int64)
        }
    )

    
    
    image = tf.decode_raw(image_features['image/encoded'], tf.uint8)
    
    # 进行reshape 操作
    image = tf.reshape(image, [image_size, image_size, 3])
    # 缩放到0-1的区间
    image = (tf.cast(image, tf.float32) - 127.5) / 128

    label = tf.cast(image_features['image/label'], tf.float32)
    image, label = tf.train.batch(
        [image, label],
        batch_size=batch_size,
        num_threads=2,
        capacity=1 * batch_size
    )
    label = tf.reshape(label, [batch_size])
    return image, label

In [12]:
import os
import tensorflow as tf
batch_size=8

image_size=28

# tf_Record路径
dataset_dir = os.path.join("dataset", 'train_data.tfrecord_shuffle')

# 读取出来的格式是tf的格式
image_batch, label_batch= read_single_tfrecord(dataset_dir,batch_size, image_size)

print("image_batch : " , image_batch)
print("label_batch : " , label_batch)


# 直接开启session
sess=tf.Session()

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

# 矩阵形式
image_batch_array,label_batch_array = sess.run([image_batch,label_batch])

print("label_batch_array:",label_batch_array)

print("image is :",image_batch_array[0].shape , image_batch_array[0])

('image_batch : ', <tf.Tensor 'batch_1:0' shape=(8, 28, 28, 3) dtype=float32>)
('label_batch : ', <tf.Tensor 'Reshape_3:0' shape=(8,) dtype=float32>)
('label_batch_array:', array([ 5.,  4.,  4.,  1.,  1.,  0.,  3.,  5.], dtype=float32))
('image is :', (28, 28, 3), array([[[-0.99609375, -0.99609375, -0.99609375],
        [-0.99609375, -0.99609375, -0.99609375],
        [-0.99609375, -0.99609375, -0.99609375],
        ..., 
        [-0.99609375, -0.99609375, -0.99609375],
        [-0.99609375, -0.99609375, -0.99609375],
        [-0.99609375, -0.99609375, -0.99609375]],

       [[-0.99609375, -0.99609375, -0.99609375],
        [-0.99609375, -0.99609375, -0.99609375],
        [-0.99609375, -0.99609375, -0.99609375],
        ..., 
        [-0.99609375, -0.99609375, -0.99609375],
        [-0.99609375, -0.99609375, -0.99609375],
        [-0.99609375, -0.99609375, -0.99609375]],

       [[-0.99609375, -0.99609375, -0.99609375],
        [-0.99609375, -0.99609375, -0.99609375],
        [-0.99609

#### 进行本地训练
- 相关依赖导入
- 模型定义
- 模型训练

In [3]:
import time
import os
import shutil
import tensorflow as tf
import numpy as np
slim = tf.contrib.slim

#####  模型定义

In [32]:
def classify_convolution(x_image,keep_prob):
    """
    模型定义:
        x_images : 输入数据  [batch_size,image_size,image_size,3] 
    """
    
    # 定义卷积
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    # 定义池化层
    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # 定义权重
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)    
    
    # 定义偏置
    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    
    # 5*5核心大小 通道数为3 输出通道数为32
    W_conv1 = weight_variable([5, 5, 3, 32])
    b_conv1 = bias_variable([32]) 
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    

    # 5*5 核心大小, 32个通道数，和上一层匹配，输出64个通道  输出 7*7*64     64 7*7
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
    
    # 链接全连接层  7*7*64个点  我们直接用1024
    W_fc1 = weight_variable([7 * 7 * 64, 1024])  # 加入一个神经元数目为1024的全连接层
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    # Dropout机制 0.7 
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
    
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    
    # 进行概率映射
    logits = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    return logits, [W_conv1, b_conv1, W_conv2, b_conv2, W_fc1, b_fc1, W_fc2, b_fc2]

##### 模型训练

In [1]:
def train_model(data_dir,max_step,batch_size,image_size=28):
    # model
    with tf.variable_scope("classify_convolution"):
        """
        定义模型输入的输出:
            输入
                x: [None 28 28 3]
                keep_prob:0.7
                
            输出:
                y:概率映射   0.1 0.8 0.0.  。。。。。。
            
        """
        input_images = tf.placeholder(tf.float32, [None, 28,28,3])  # 将某些特殊的操作指定为 "feed" 操作, 标记的方法是使用 tf.placeholder() 为这些操作创建占位符.
        keep_prob = tf.placeholder(tf.float32)
        logits, variables = classify_convolution(input_images, keep_prob)


    # 这个就是label输入
    labels = tf.placeholder(tf.float32, [None, 10])
    
    
    # 计算网络输出和label的差距，并使用优化器最小化差距，Adam
    loss = -tf.reduce_sum(labels * tf.log(logits))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
    
    
    # 计算accuracy
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # 进行参数初始化
    saver = tf.train.Saver(variables)
    init = tf.global_variables_initializer()
    
    
    label_batch_array=None
    image_batch_array=None
    
    #  进行数据的读取
    image_batch, label_batch= read_single_tfrecord(data_dir, batch_size, image_size)
    
    with tf.Session() as sess:
        # 初始化操作
        sess.run(init)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        for i in range(max_step):
            
            # 进行数据和标签转换
            image_batch_array,label_batch_one = sess.run([image_batch,label_batch])
            label_batch_one=np.array(label_batch_one,np.int16)
            label_batch_array=np.zeros((batch_size,10))
            label_batch_array[np.arange(batch_size),label_batch_one]=1
            
            
            # 每100次进行测试一次
            if i % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={input_images: image_batch_array, labels: label_batch_array, keep_prob: 0.7})
                
                print("step %d, training accuracy %g" % (i, train_accuracy))
                
            # 开启训练    
            sess.run(train_step, feed_dict={input_images: image_batch_array, labels: label_batch_array, keep_prob: 0.7})
            
        path = saver.save(sess, os.path.join(os.path.dirname(__file__), "model/convolutional.ckpt"))
        print("Saved:", path)
        
def test__model(model_path,test_images):
    """
    
    """

In [1]:
max_step=50000 #最大训练次数
batch_size=32  #设置batch_size
image_size=28

train_model(os.path.join("dataset", 'train_data.tfrecord_shuffle'),50000,32,28)

NameError: name 'train_model' is not defined

## TaaS模型训练与测试
- 

##### 模型定义

In [ ]:
def classify_convolution(x_image,keep_prob):
    """
    模型定义:
        x_images : 输入数据  [batch_size,image_size,image_size,3] 
    """
    
    # 定义卷积
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    # 定义池化层
    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # 定义权重
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)    
    
    # 定义偏置
    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    
    # 5*5核心大小 通道数为3 输出通道数为32
    W_conv1 = weight_variable([5, 5, 3, 32])
    b_conv1 = bias_variable([32]) 
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    

    # 5*5 核心大小, 32个通道数，和上一层匹配，输出64个通道  输出 7*7*64     64 7*7
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
    
    # 链接全连接层  7*7*64个点  我们直接用1024
    W_fc1 = weight_variable([7 * 7 * 64, 1024])  # 加入一个神经元数目为1024的全连接层
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    # Dropout机制 0.7 
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
    
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    
    # 进行概率映射
    logits =tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    return logits, [W_conv1, b_conv1, W_conv2, b_conv2, W_fc1, b_fc1, W_fc2, b_fc2]

In [ ]:
##### TaaS平台训练模板
# coding=utf-8
import tensorflow as tf
from caicloud.clever.tensorflow import dist_base
from tensorflow.python.ops import control_flow_ops
import tensorflow.contrib.slim as slim
import datasets_factory


# 定义 Tensorflow Flags, 类似于 python argparse.
tf.app.flags.DEFINE_boolean('random_flip_up_down', False, "Whether to random flip up down")
tf.app.flags.DEFINE_boolean('random_brightness', True, "whether to adjust brightness")
tf.app.flags.DEFINE_boolean('random_contrast', True, "whether to random constrast")

tf.app.flags.DEFINE_integer('charset_size', 3755, "Choose the first `charset_size` characters only.")
tf.app.flags.DEFINE_integer('image_size', 28, "Needs to provide same value as in training.")
tf.app.flags.DEFINE_boolean('gray', True, "whether to change the rbg to gray")
tf.app.flags.DEFINE_integer('max_steps', 60000, 'the max training steps ')
tf.app.flags.DEFINE_integer('eval_steps', 50, "the step num to eval")
tf.app.flags.DEFINE_integer('save_steps', 500, "the steps to save")

tf.app.flags.DEFINE_string('checkpoint_dir', 'checkpoint/', 'the checkpoint dir')
tf.app.flags.DEFINE_string('train_data_dir', 'dataset/', 'the train dataset dir')
tf.app.flags.DEFINE_string('test_data_dir', 'dataset/', 'the test dataset dir')
tf.app.flags.DEFINE_string('log_dir', 'ocr_log', 'the logging dir')

tf.app.flags.DEFINE_boolean('restore', False, 'whether to restore from checkpoint')
tf.app.flags.DEFINE_boolean('epoch', 1, 'Number of epochs')
tf.app.flags.DEFINE_integer('batch_size', 64, 'Batch size')
tf.app.flags.DEFINE_string('mode', 'train', 'Running mode. One of {"train", "valid", "test"}')
tf.app.flags.DEFINE_string('test_image', 'data/test.png', 'image path to test the model')
tf.app.flags.DEFINE_integer('num_readers', 4, 'The number of parallel readers that read data from the dataset.')


# 通过这句来加载 Flags
FLAGS = tf.app.flags.FLAGS


# 定义一些全局变量，在函数之间调用
keep_prob = None   # dropout 开启神经元概率
train_op = None   # slim训练操作变量
accuracy = None   # 准确率
global_step = None   # 全局步数

predicted_val_top_k = None   # top k 的预测概率
predicted_index_top_k = None   # top k 的预测index
merged_summary_op = None   # tf summary 合并记录
accuracy_in_top_k = None   # top k 准确率


# 定义 Tensorflow 模型函数
def model_fn(sync, num_replicas):
    # 从全局读取变量
    global keep_prob,  train_op, accuracy, global_step

    # 创建slim全局步数变量
    global_step = slim.create_global_step()

    # 读取 TFRecord 文件，获取训练数据队列
    dataset = datasets_factory.get_split("main_train",FLAGS.train_data_dir)
    provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset,
        num_readers=FLAGS.num_readers,
        common_queue_capacity=20 * FLAGS.batch_size,
        common_queue_min=10 * FLAGS.batch_size)
    [image, label] = provider.get(['image', 'label'])

    # 图像增强
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.random_brightness(image, max_delta=0.3)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    new_size = tf.constant([FLAGS.image_size, FLAGS.image_size], dtype=tf.int32)
    image = tf.image.resize_images(image, new_size)

    # 调用 tf.train.batch 来获取 batch 大小的数据流
    images, labels = tf.train.batch(
        [image, label],
        batch_size=FLAGS.batch_size,
        num_threads=4,
        capacity=5 * FLAGS.batch_size
    )

    # 定义 tensorflow 占位符
    keep_prob = tf.placeholder(dtype=tf.float32, shape=[], name='keep_prob')

    
        
    logits,_=classify_convolution(images,keep_prob)

    # 定义loss
    
    
    loss = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels))

    
    # 定义Update ops
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    if update_ops:
        updates = tf.group(*update_ops)
        loss = control_flow_ops.with_dependencies([updates], loss)

    # 定义优化器，也就是反向传播参数更新过程，使用了AdamOptimizer，可以修改成别的。
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

    # 是否同步更新参数，分布式系统使用
    if sync:
        num_workers = num_replicas
        optimizer = tf.train.SyncReplicasOptimizer(
            optimizer,
            replicas_to_aggregate=num_workers,
            total_num_replicas=num_workers,
            name="ocr_sync_replicas")

    # 创建 slim 训练变量
    train_op = slim.learning.create_train_op(loss, optimizer, global_step=global_step)

    # 定义准确率
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), labels), tf.float32))

    # 返回 dist_base
    return dist_base.ModelFnHandler(
        global_step=global_step,
        optimizer=optimizer)


local_step = 0


# 定义模型训练函数
def train_fn(session, num_global_step):
    global keep_prob,  train_op, accuracy, local_step, global_step

    # validation 测试
    if local_step % FLAGS.eval_steps == 1:
        val_dict = {keep_prob: 1.0}

        validate_acc = session.run(
            accuracy,
            feed_dict=val_dict)
        print("After %d training step(s), validation accuracy is %g " % (
            num_global_step, validate_acc))

    local_step += 1

    train_dict = {keep_prob: 0.8,}

    # 训练步
    total_loss, cur_global_step = session.run([train_op, global_step], feed_dict=train_dict)

    # 每20步打印一次结果
    if local_step % 20 == 0:
        print('local step: {0}, global step {1}: '
              'loss = {2:.4f}'.format(local_step, cur_global_step, total_loss))
    
    return False


# 定义训练结束后的评测函数
def after_train_hook(sess):
    global  keep_prob, accuracy

    test_feed = {keep_prob: 1.0}

    test_acc = sess.run(accuracy, feed_dict=test_feed)
    print("Test accuracy is %g" % test_acc)


# 主函数
if __name__ == '__main__':
    distTfRunner = dist_base.DistTensorflowRunner(
        model_fn=model_fn,
        after_train_hook=after_train_hook)
    distTfRunner.run(train_fn)